In [ ]:
import os
import sys
from pathlib import Path

os.chdir(Path(os.getcwd()).parents[0])
sys.path.append(os.getcwd())

import glob
import numpy as np
import torch
import pandas as pd

from scipy.stats import pearsonr
from scipy.stats import sem

from src.utils.plot_utils import *


def NormalizeData(data, min, max):
    return (data - min) / ((max - min) + 0.00000000001)

### Import Evaluation Scores

In [ ]:
# File loading per dataset

file_image_inet = "/image/eval_scores_imagenet.npz"
file_image_oct = "/image/eval_scores_oct.npz"
file_image_r45 = "/image/eval_scores_resisc45.npz"

file_volume_adr = "/volume/eval_scores_adrenalmnist3d.npz"
file_volume_org = "/volume/eval_scores_organmnist3d.npz"
file_volume_ves = "/volume/eval_scores_vesselmnist3d.npz"

file_pc_coma = "/point_cloud/eval_scores_coma.npz"
file_pc_m40 = "/point_cloud/eval_scores_modelnet40.npz"
file_pc_shpn = "/point_cloud/eval_scores_shapenet.npz"

file_loc = os.getcwd() + "/data/evaluation_scores"

file = np.load(file_loc + file_image_inet, allow_pickle=True)
arr_image_inet = [file["arr_0"], file["arr_1"], file["arr_2"]]
file = np.load(file_loc + file_image_oct, allow_pickle=True)
arr_image_oct = [file["arr_0"], file["arr_1"], file["arr_2"]]
file = np.load(file_loc + file_image_r45, allow_pickle=True)
arr_image_r45 = [file["arr_0"], file["arr_1"], file["arr_2"]]

file = np.load(file_loc + file_volume_adr, allow_pickle=True)
arr_volume_adr = [file["arr_0"], file["arr_1"], file["arr_2"]]
file = np.load(file_loc + file_volume_org, allow_pickle=True)
arr_volume_org = [file["arr_0"], file["arr_1"], file["arr_2"]]
file = np.load(file_loc + file_volume_ves, allow_pickle=True)
arr_volume_ves = [file["arr_0"], file["arr_1"], file["arr_2"]]

file = np.load(file_loc + file_pc_coma, allow_pickle=True)
arr_pc_coma = [file["arr_0"], file["arr_1"], file["arr_2"]]
file = np.load(file_loc + file_pc_m40, allow_pickle=True)
arr_pc_m40 = [file["arr_0"], file["arr_1"], file["arr_2"]]
file = np.load(file_loc + file_pc_shpn, allow_pickle=True)
arr_pc_shpn = [file["arr_0"], file["arr_1"], file["arr_2"]]

### Ranking Computation

#### Full Ranking

In [ ]:
arr_image = [arr_image_inet, arr_image_oct, arr_image_r45]
arr_volume = [arr_volume_adr, arr_volume_org, arr_volume_ves]
arr_pc = [arr_pc_coma, arr_pc_m40, arr_pc_shpn]
arr_modalities = [arr_image, arr_volume, arr_pc]

arr_ranking = np.empty(
    [3, 3, 3, 17, 20], dtype=float
)  # modality, dataset, model, xai, eval
arr_ranking[:] = np.nan

bup_order = [0, 1, 2, 4, 5, 7, 9, 12, 17]

for modality in range(3):
    for dataset in range(3):
        for model in range(3):
            for xai in range(arr_modalities[modality][dataset][model].shape[0]):
                for eval in range(20):
                    ranking = np.median(
                        arr_modalities[modality][dataset][model][:, eval, :], -1
                    ).argsort()  # compute ranking based on median obs score
                    if eval in bup_order:
                        ranking = ranking[
                            ::-1
                        ]  # reverse ranking to bottom up if larger is better

                    pos = (
                        ranking.argsort()[xai] + 1
                    )  # get rankin position of xai method (+1 so ranking starts at 1 and not 0)
                    arr_ranking[modality, dataset, model, xai, eval] = pos

arr_table = []
for eval in range(20):
    for modality in range(3):
        for dataset in range(3):
            arr_col_val = []
            for model in [2]:
                for xai in range(17):
                    if modality == 2 and xai == 6:
                        arr_col_val = arr_col_val + [
                            np.round(np.mean(arr_ranking[(0, 1), :, :, 6, eval])),
                            np.round(np.mean(arr_ranking[(0, 1), :, :, 7, eval])),
                            np.round(np.mean(arr_ranking[(0, 1), :, :, 8, eval])),
                        ]
                    if modality == 2 and xai == 11:
                        break
                    x = arr_ranking[modality, dataset, model, xai, eval]
                    val = np.round(np.mean(x[~np.isnan(x)]))
                    arr_col_val.append(val)
                arr_table.append(arr_col_val)

df_table = pd.DataFrame(arr_table).transpose()
df_table.index = [
    "OC",
    "LIME",
    "KS",
    "VG",
    "IxG",
    "GB",
    "GC",
    "SC",
    "C+",
    "IG",
    "EG",
    "DL",
    "DLS",
    "LRP",
    "RA",
    "RoA",
    "LA",
]

#### Ranking across Model Architectures

In [ ]:
arr_image = [arr_image_inet, arr_image_oct, arr_image_r45]
arr_volume = [arr_volume_adr, arr_volume_org, arr_volume_ves]
arr_pc = [arr_pc_coma, arr_pc_m40, arr_pc_shpn]
arr_modalities = [arr_image, arr_volume, arr_pc]

arr_ranking = np.empty([3, 17, 20], dtype=float)  # modality, dataset, xai, eval
arr_ranking[:] = np.nan

bup_order = [0, 1, 2, 4, 5, 7, 9, 12, 17]

for modality in range(3):
    for eval in range(20):
        arr_scores = []
        for model in range(3):
            for data in range(3):
                d = arr_modalities[modality][data][model][:, eval, :]
                q_h = np.quantile(d, 0.975)
                q_l = np.quantile(d, 0.025)

                d = np.clip(d, q_l, q_h)
                d_max = d.max()
                d_min = d.min()
                arr_scores.append(NormalizeData(d, d_min, d_max))

        model_1 = np.column_stack(
            (
                np.median(arr_scores[0], 1),
                np.median(arr_scores[1], 1),
                np.median(arr_scores[2], 1),
            )
        )
        model_2 = np.column_stack(
            (
                np.median(arr_scores[3], 1),
                np.median(arr_scores[4], 1),
                np.median(arr_scores[5], 1),
            )
        )
        model_3 = np.column_stack(
            (
                np.median(arr_scores[6], 1),
                np.median(arr_scores[7], 1),
                np.median(arr_scores[8], 1),
            )
        )

        ranking = np.concatenate(
            [
                np.mean(
                    np.hstack([model_1, model_2, model_3[:-3]]),
                    -1,
                ),
                np.mean(model_3[-3:], -1),
            ]
        ).argsort()
        # compute ranking based on median obs score
        if eval in bup_order:
            ranking = ranking[::-1]  # reverse ranking to bottom up if larger is better

        for xai in range(ranking.shape[0]):
            pos = (
                ranking.argsort()[xai] + 1
            )  # get rankin position of xai method (+1 so ranking starts at 1 and not 0)
            arr_ranking[modality, xai, eval] = pos

### Tables as in Paper

#### Faithfulness

In [ ]:
img_faith = arr_ranking[0, :, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16]].T.round(1)
img_faith_table = np.column_stack(
    (img_faith.mean(1).round(1), img_faith.std(1).round(1), img_faith)
)
np.savetxt("./img_faith.csv", img_faith_table, delimiter=",")

vol_faith = arr_ranking[1, :, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16]].T.round(1)
vol_faith_table = np.column_stack(
    (vol_faith.mean(1).round(1), vol_faith.std(1).round(1), vol_faith)
)
np.savetxt("./vol_faith.csv", vol_faith_table, delimiter=",")

pc_faith = arr_ranking[2, :, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16]].T.round(1)
pc_faith_table = np.column_stack(
    (pc_faith.mean(1).round(1), pc_faith.std(1).round(1), pc_faith)
)
np.savetxt("./pc_faith.csv", pc_faith_table, delimiter=",")

#### Robustness

In [ ]:
img_rob = arr_ranking[0, :, [10, 11, 12, 13, 14, 15]].T.round(1)
img_rob_table = np.column_stack(
    (img_rob.mean(1).round(1), img_rob.std(1).round(1), img_rob)
)
np.savetxt("./img_rob.csv", img_rob_table, delimiter=",")

vol_rob = arr_ranking[1, :, [10, 11, 12, 13, 14, 15]].T.round(1)
vol_rob_table = np.column_stack(
    (vol_rob.mean(1).round(1), vol_rob.std(1).round(1), vol_rob)
)
np.savetxt("./vol_rob.csv", vol_rob_table, delimiter=",")

pc_rob = arr_ranking[2, :, [10, 11, 12, 13, 14, 15]].T.round(1)
pc_rob_table = np.column_stack(
    (pc_rob.mean(1).round(1), pc_rob.std(1).round(1), pc_rob)
)
np.savetxt("./pc_rob.csv", pc_rob_table, delimiter=",")

#### Complexity

In [ ]:
img_comp = arr_ranking[0, :, [17, 18, 19]].T.round(1)
img_comp_table = np.column_stack(
    (img_comp.mean(1).round(1), img_comp.std(1).round(1), img_comp)
)
np.savetxt("./img_comp.csv", img_comp_table, delimiter=",")

vol_comp = arr_ranking[1, :, [17, 18, 19]].T.round(1)
vol_comp_table = np.column_stack(
    (vol_comp.mean(1).round(1), vol_comp.std(1).round(1), vol_comp)
)
np.savetxt("./vol_comp.csv", vol_comp_table, delimiter=",")

pc_comp = arr_ranking[2, :, [17, 18, 19]].T.round(1)
pc_comp_table = np.column_stack(
    (pc_comp.mean(1).round(1), pc_comp.std(1).round(1), pc_comp)
)
np.savetxt("./pc_comp.csv", pc_comp_table, delimiter=",")

### MDS Scaling Plot

In [ ]:
arr_table = []
for eval in [(0, 10), (10, 17), (17, 20)]:
    for modality in range(3):
        arr_col_val = []
        arr_col_std = []
        for xai in range(17):
            if modality == 2 and xai == 6:
                arr_col_val = arr_col_val + [np.nan, np.nan, np.nan]
            if modality == 2 and xai == 14:
                break
            x = arr_ranking[modality, :, xai, eval[0] : eval[1]]
            val = np.round(np.mean(x[~np.isnan(x)]))
            if not np.isnan(val):
                val = int(val)
            else:
                val = np.nan
            arr_col_val.append(val)
        arr_table.append(arr_col_val)

df_table = pd.DataFrame(arr_table).transpose()
df_table.index = [
    "OC",
    "LI",
    "KS",
    "VG",
    "IxG",
    "GB",
    "GC",
    "SC",
    "C+",
    "IG",
    "EG",
    "DL",
    "DLS",
    "LRP",
    "RA",
    "RoA",
    "LA",
]

In [ ]:
from sklearn.manifold import MDS, TSNE

mds = MDS(n_components=2, random_state=4)
# mds = TSNE(perplexity= 10)
X_transformed = mds.fit_transform(df_table)

import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.G10

fig = go.Figure()

X_transformed[4, 0] = X_transformed[4, 0] - 4

fig.add_trace(
    go.Scatter(
        x=X_transformed[:-3, 0],
        y=X_transformed[:-3, 1],
        mode="markers+text",
        text=[
            "OC",
            "LIME",
            "KS",
            "VG",
            "IxG",
            "GB",
            "GC",
            "SC",
            "C+",
            "IG",
            "EG",
            "DL",
            "DLS",
            "LRP",
        ],
        textposition="top right",
        name="Attribution",
        marker=dict(color=colors[0], size=8),
    )
)

fig.add_trace(
    go.Scatter(
        x=X_transformed[-3:, 0],
        y=X_transformed[-3:, 1],
        mode="markers+text",
        text=["RA", "RoA", "LA"],
        textposition="top right",
        name="Attention",
        marker=dict(color=colors[2], size=8),
    )
)

fig.update_layout(
    height=500,
    width=550,
    xaxis=dict(
        title="Dim 1",
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title="Dim 2",
        titlefont_size=16,
        tickfont_size=14,
    ),
    font=dict(family="Helvetica", color="#000000", size=14),
    legend_title=dict(
        text="XAI Methods", font=dict(family="Helvetica", size=16, color="#000000")
    ),
    template="plotly_white",
)

# fig.write_image(os.getcwd() + "/data/figures/mds_plot.png", scale=2)
fig.show()

### Avg. Rank per XAI Method

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.G10
fig = go.Figure()

x = [
    [
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attribution",
        "Attention",
        "Attention",
        "Attention",
    ],
    [
        "OC",
        "LI",
        "KS",
        "VG",
        "IxG",
        "GB",
        "GC",
        "SC",
        "C+",
        "IG",
        "EG",
        "DL",
        "DLS",
        "LRP",
        "RA",
        "RoA",
        "LA",
    ],
]
fig.add_trace(
    go.Bar(
        x=x,
        y=np.round(np.mean(df_table.iloc[:, :90], axis=1), 1),
        name="Faithfullness",
        marker_color=colors[0],
        textposition="inside",
        insidetextanchor="start",
        text=np.round(np.mean(df_table.iloc[:, :90], axis=1), 1),
        error_y=dict(
            type="data", array=np.round(sem(df_table.iloc[:, :90], axis=1), 2)
        ),
    )
)

fig.add_trace(
    go.Bar(
        x=x,
        y=np.round(np.mean(df_table.iloc[:, 90:153], axis=1), 1),
        name="Robustness",
        marker_color=colors[4],
        textposition="inside",
        insidetextanchor="start",
        text=np.round(np.mean(df_table.iloc[:, 90:153], axis=1), 1),
        error_y=dict(
            type="data", array=np.round(sem(df_table.iloc[:, 90:153], axis=1), 2)
        ),
    )
)

fig.add_trace(
    go.Bar(
        x=x,
        y=np.round(np.mean(df_table.iloc[:, 153:180], axis=1), 1),
        name="Complexity",
        marker_color=colors[5],
        textposition="inside",
        insidetextanchor="start",
        text=np.round(np.mean(df_table.iloc[:, 153:180], axis=1), 1),
        error_y=dict(
            type="data", array=np.round(sem(df_table.iloc[:, 153:180], axis=1), 2)
        ),
    )
)

fig.update_yaxes(zerolinewidth=4)

fig.update_layout(
    xaxis=dict(
        title="XAI Methods",
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title="Average Rank",
        titlefont_size=16,
        tickfont_size=14,
    ),
    font=dict(family="Helvetica", color="#000000", size=14),
    legend_title=dict(
        text="Evaluation Criteria",
        font=dict(family="Helvetica", size=16, color="#000000"),
    ),
    barmode="group",
    template="plotly_white",
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.05,  # gap between bars of the same location coordinate.
    height=500,
    width=1500,
)

# fig.write_image(os.getcwd() + "/data/figures/aa_full_plot.png", scale=2)
fig.show()

#### Correltation Matrix

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.G10


df_corr = np.corrcoef(df_table)
mask = np.ones_like(df_corr, dtype=bool)
mask[:] = np.nan
mask = np.triu(mask).T


heat = go.Heatmap(
    z=df_corr * mask,
    x=df_table.index,
    y=df_table.index,
    text=df_corr * mask,
    zmin=-1,  # Sets the lower bound of the color domain
    zmax=1,
    xgap=1,  # Sets the horizontal gap (in pixels) between bricks
    ygap=1,
    colorscale=[
        [0, "#B5545C"],
        [0.1, "#B5545C"],
        [0.5, "rgb(245, 245, 245)"],
        [0.9, "#76BB40"],
        [1, "#76BB40"],
    ],
    coloraxis_colorbar=dict(
        thicknessmode="pixels",
        thickness=10,
        lenmode="pixels",
        len=10,
    ),
)

layout = go.Layout(
    width=600,
    height=600,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange="reversed",
    xaxis=dict(
        title="XAI Methods",
        titlefont_size=18,
        tickfont_size=16,
    ),
    yaxis=dict(
        title="XAI Methods",
        titlefont_size=18,
        tickfont_size=16,
    ),
    font=dict(family="Helvetica", color="#000000", size=16),
)

fig = go.Figure(data=[heat], layout=layout)

# fig.write_image(os.getcwd().split("src")[0] + "data/figures/corr_total.png", scale=2)
fig.show()

### Average Rank per Evaluation Criterion

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.G10
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=["Faithfullness", "Robustness", "Complexity"],
        y=[
            np.round(np.nanmean(df_table.iloc[:-3, :90]), 1),
            np.round(np.nanmean(df_table.iloc[:-3, 90:153]), 1),
            np.round(np.nanmean(df_table.iloc[:-3, 153:180]), 1),
        ],
        name="Attribution",
        marker_color=colors[0],
        textposition="inside",
        insidetextanchor="start",
        text=[
            np.round(np.nanmean(df_table.iloc[:-3, :90]), 1),
            np.round(np.nanmean(df_table.iloc[:-3, 90:153]), 1),
            np.round(np.nanmean(df_table.iloc[:-3, 153:180]), 1),
        ],
        error_y=dict(
            type="data",
            array=[
                sem(df_table.iloc[:-3, :90], axis=None),
                sem(df_table.iloc[:-3, 90:153], axis=None),
                sem(df_table.iloc[:-3, 153:180], axis=None),
            ],
        ),
    )
)

fig.add_trace(
    go.Bar(
        x=["Faithfullness", "Robustness", "Complexity"],
        y=[
            np.round(np.nanmean(df_table.iloc[-3:, :90]), 1),
            np.round(np.nanmean(df_table.iloc[-3:, 90:153]), 1),
            np.round(np.nanmean(df_table.iloc[-3:, 153:180]), 1),
        ],
        name="Attention",
        marker_color=colors[2],
        textposition="inside",
        insidetextanchor="start",
        text=[
            np.round(np.nanmean(df_table.iloc[-3:, :90]), 1),
            np.round(np.nanmean(df_table.iloc[-3:, 90:153]), 1),
            np.round(np.nanmean(df_table.iloc[-3:, 153:180]), 1),
        ],
        error_y=dict(
            type="data",
            array=[
                sem(df_table.iloc[15:, :90], axis=None),
                sem(df_table.iloc[15:, 90:153], axis=None),
                sem(df_table.iloc[15:, 153:180], axis=None),
            ],
        ),
    )
)

fig.update_yaxes(zerolinewidth=4)

fig.update_layout(
    xaxis=dict(
        title="Evaluation Criteria",
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title="Average Rank",
        titlefont_size=16,
        tickfont_size=14,
    ),
    font=dict(family="Helvetica", color="#000000", size=14),
    legend_title=dict(
        text="XAI Methods", font=dict(family="Helvetica", size=16, color="#000000")
    ),
    barmode="group",
    template="plotly_white",
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.05,  # gap between bars of the same location coordinate.
    height=500,
    width=700,
)

from scipy import stats

t1 = stats.ttest_ind(
    a=df_table.iloc[:-3, :90].to_numpy().flatten(),
    b=df_table.iloc[-3:, :90].to_numpy().flatten(),
    equal_var=False,
)[1]
t2 = stats.ttest_ind(
    a=df_table.iloc[:-3, 90:153].to_numpy().flatten(),
    b=df_table.iloc[-3:, 90:153].to_numpy().flatten(),
    equal_var=False,
)[1]
t3 = stats.ttest_ind(
    a=df_table.iloc[:-3, 153:180].to_numpy().flatten(),
    b=df_table.iloc[-3:, 153:180].to_numpy().flatten(),
    equal_var=False,
)[1]

for i in range(3):
    fig = add_p_value_annotation(
        fig,
        array_columns=[[-0.25 + i, 0.25 + i]],
        p_value=[[t1, t2, t3][i]],
        _format=dict(interline=0.06, text_height=1.08, color="black"),
    )

# fig.write_image(os.getcwd() + "/data/figures/aa_aggr_plot.png", scale=2)
fig.show()

### Modality Plot

In [ ]:
arr_table = []
for modality in range(3):
    for eval in range(20):
        for dataset in range(3):
            arr_col_val = []
            for xai in range(17):
                if modality == 2 and xai == 6:
                    arr_col_val = arr_col_val + [np.nan, np.nan, np.nan]
                if modality == 2 and xai == 14:
                    break
                val = arr_ranking[modality, dataset, xai, eval]
                arr_col_val.append(val)
            arr_table.append(arr_col_val)

df_table = pd.DataFrame(arr_table).transpose()
df_table.index = [
    "OC",
    "LI",
    "KS",
    "SA",
    "IxG",
    "GB",
    "GC",
    "SC",
    "C+",
    "IG",
    "EG",
    "DL",
    "DLS",
    "LRP",
    "RA",
    "RoA",
    "LA",
]

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.G10
fig = go.Figure()

x1 = ["Faithfullness"] * 4 + ["Robustness"] * 4 + ["Complexity"] * 4
x2 = ["LI", "GC", "EG", "LA"] * 3
x = [x1, x2]

fig.add_trace(
    go.Bar(
        x=x,
        y=np.concatenate(
            (
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], :30], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 30:51], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 51:60], axis=1), 1),
            ),
            axis=None,
        ),
        name="Image",
        marker_color=colors[0],
        textposition="inside",
        insidetextanchor="start",
        text=np.concatenate(
            (
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], :30], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 30:51], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 51:60], axis=1), 1),
            ),
            axis=None,
        ),
        error_y=dict(
            type="data",
            array=np.concatenate(
                (
                    sem(df_table.iloc[[1, 6, 10, 16], :30], axis=1),
                    sem(df_table.iloc[[1, 6, 10, 16], 30:51], axis=1),
                    sem(df_table.iloc[[1, 6, 10, 16], 51:60], axis=1),
                ),
                axis=None,
            ),
        ),
    )
)


fig.add_trace(
    go.Bar(
        x=x,
        y=np.concatenate(
            (
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 60:90], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 90:111], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 111:120], axis=1), 1),
            ),
            axis=None,
        ),
        name="Volume",
        marker_color=colors[3],
        textposition="inside",
        insidetextanchor="start",
        text=np.concatenate(
            (
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 60:90], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 90:111], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 111:120], axis=1), 1),
            ),
            axis=None,
        ),
        error_y=dict(
            type="data",
            array=np.concatenate(
                (
                    sem(df_table.iloc[[1, 6, 10, 16], 60:90], axis=1),
                    sem(df_table.iloc[[1, 6, 10, 16], 90:111], axis=1),
                    sem(df_table.iloc[[1, 6, 10, 16], 111:120], axis=1),
                ),
                axis=None,
            ),
        ),
    )
)

fig.add_trace(
    go.Bar(
        x=x,
        y=np.concatenate(
            (
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 120:150], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 150:171], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 171:180], axis=1), 1),
            ),
            axis=None,
        ),
        name="Point Cloud",
        marker_color=colors[5],
        textposition="inside",
        insidetextanchor="start",
        text=np.concatenate(
            (
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 120:150], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 150:171], axis=1), 1),
                np.round(np.mean(df_table.iloc[[1, 6, 10, 16], 171:180], axis=1), 1),
            ),
            axis=None,
        ),
        error_y=dict(
            type="data",
            array=np.concatenate(
                (
                    sem(df_table.iloc[[1, 6, 10, 16], 120:150], axis=1),
                    sem(df_table.iloc[[1, 6, 10, 16], 150:171], axis=1),
                    sem(df_table.iloc[[1, 6, 10, 16], 171:180], axis=1),
                ),
                axis=None,
            ),
        ),
    )
)

fig.update_yaxes(zerolinewidth=4)

fig.update_layout(
    xaxis=dict(
        title="XAI Methods per Evaluation Criteria",
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title="Average Rank",
        titlefont_size=16,
        tickfont_size=14,
    ),
    font=dict(family="Helvetica", color="#000000", size=14),
    legend_title=dict(
        text="Modality", font=dict(family="Helvetica", size=16, color="#000000")
    ),
    barmode="group",
    template="plotly_white",
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.05,  # gap between bars of the same location coordinate.
    height=500,
    width=1500,
)

# fig.write_image(os.getcwd() + "/data/figures/mod_aggr_plot.png", scale=2)
fig.show()